import library


In [35]:
import openai
import json
import jsonlines
import time 


import data

In [36]:
# path to training data
data_path = './data/train/train_data.jsonl '

# read data in training data
#training_data = []

with open(data_path) as data:
    dataset = [json.loads(line) for line in data]

fine_tune_data = [{'id': example['id'], 'role': 'user', 'Problem': example['Problem'], 'options': example['options'], 'diagramRef': example['diagramRef'], 'correct': example['correct']} for example in dataset]


setup dataset

In [37]:
dataset[0]

{'id': 14782,
 'Problem': 'a man whose bowling average is 12.4 , takes 5 wickets for 26 runs and there by decreases his average by 0.4 . the number of wickets taken by him before his last match is ?',
 'options': 'a ) 82 , b ) 83 , c ) 84 , d ) 85 , e ) 86',
 'Rationale': '"12.4 x + 26 / ( 5 + x ) = 12 12.4 x + 26 = 60 + 12 x x = 34 / 0.4 = 85 answer : d"',
 'correct': 'd',
 'diagramRef': '',
 'category': 'general'}

In [38]:
fine_tune_data[0]

{'id': 14782,
 'role': 'user',
 'Problem': 'a man whose bowling average is 12.4 , takes 5 wickets for 26 runs and there by decreases his average by 0.4 . the number of wickets taken by him before his last match is ?',
 'options': 'a ) 82 , b ) 83 , c ) 84 , d ) 85 , e ) 86',
 'diagramRef': '',
 'correct': 'd'}

In [39]:
def read_data(fine_tune_data):
  for example in fine_tune_data:
    if example['options'] != "":
      example.update(problem =  example['Problem'] + "\n options: " + example['options'])
    else:
      example.update(problem =  example['Problem'] + "\n" + "the answer of the question is: ")
    example.pop('Problem',None)
    example.pop('options',None)
read_data(fine_tune_data)

In [51]:
fine_tune_data[6:9]

[{'id': 14788,
  'role': 'user',
  'diagramRef': '',
  'correct': 'd',
  'problem': 'a bag contains 6 white marbles and 6 black marbles . if each of 6 girls and 6 boys randomly selects and keeps a marble , what is the probability that all of the girls select the same colored marble ?\n options: a ) 1 / 35 , b ) 1 / 10 , c ) 1 / 3 , d ) 1 / 462 , e ) 1 / 252'},
 {'id': 14789,
  'role': 'user',
  'diagramRef': '',
  'correct': 'a',
  'problem': 'john want to buy a $ 100 trouser at the store , but he think it ’ s too expensive . finally , it goes on sale for $ 10 . what is the percent decrease ?\n options: a ) 90 % , b ) 30 % , c ) 40 % , d ) 70 % , e ) 80 %'},
 {'id': 14790,
  'role': 'user',
  'diagramRef': '',
  'correct': 'd',
  'problem': 'if a train , traveling at a speed of 270 kmph , crosses a pole in 5 sec , then the length of train is ?\n options: a ) 281 m , b ) 325 m , c ) 117 m , d ) 375 m , e ) 350 m'}]

upload API and function set up input and output

In [41]:
from openai import OpenAI
with open('./data/train/keyopenai.txt') as f:
    api_key = f.readline().strip()
    client = OpenAI(api_key=api_key)

In [42]:
from openai.types import Completion, CompletionChoice, CompletionUsage

def get_response(user_request):
    """function get an response from API

    Args:
        user_request (string): a question about SAT 

    Returns:
        response: answer
    """
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt = user_request,
        max_tokens = 20,
        n = 1,
        temperature = 0.2
    )
    return response.choices[0].text

test function

In [43]:
question =' what is 1 add 1 equal to?'
answer = get_response(question)
print(answer)



1 add 1 is equal to 2.


In [54]:
from typing import List
def convert_to_submit_file(api_result: list = []):
    """function write the results in file results.txt 

    Args:
        api_result (list, optional): results of function get_response

    Returns:
        string: save results in a text file
    """
    api_result_str = ' '.join(api_result)

    answer_start = api_result_str.find("Answer: ")
    if answer_start != -1:
        answer_end = api_result_str.find(",", answer_start)
        answer_part = api_result_str[answer_start + len("Answer: "):answer_end]

        if any(c.isalpha() for c in answer_part):
            answer = answer_part[0:answer_part.find(")")]
        else:
            answer = answer_part
        return answer.lower()
    else:
        return api_result_str.lower()

solution


In [55]:
#check current_id 
result_file_path ='./results/results.txt'
def check_current_id(result_file_path) -> int:
    with open(result_file_path, 'r') as f:
        lines = f.readlines()
        current_id = len(lines)
    return current_id


In [56]:
quantityOfCorrectAnswer=0
def main():
    current_id =check_current_id(result_file_path)
    with open(result_file_path, 'a') as f:
        print("open result file ")
        for example in fine_tune_data[current_id-1:]:
            prompt ="You are a helpful assistant that answers math questions. You should answer the questions in the following format:For multiple-choice questions without an answer: Only provide the final result. Question: What is the result of the calculation 3+6? 9 .Question: What is the result of the calculation 4+2-6? 0 .For multiple-choice questions with the correct answer: Only provide the final result.Question: What is the result of the calculation 3+4+7? a)14 b)6 c)34  a .Question: What is the sum of 2 and 9? a)4 b)5 c)11  c .For multiple-choice questions with special characters such as $, kg, meter in the answer: Only print the numerical result.Question: How much is 1 kg plus 1 kg?  2 .Question: How much is $10 plus $200? a)$300 b)$210  b  \nQuestion:"
            user_request= prompt + example['problem']
            try:   
                t1 =time.time()
                answer_from_api =get_response(user_request)
                t2= time.time()
                time_request=t2 - t1
                print("success")
            except:
                print("failed")
                continue
            print(f"Time request for {example['id']}: {time_request}, answer: {answer_from_api}")
            fill_in_file =convert_to_submit_file(answer_from_api)
            
            if fill_in_file == example["correct"]:
                quantityOfCorrectAnswer = quantityOfCorrectAnswer + 1
            f.write( fill_in_file + "\t"+str(time_request) + "\n")
            time.sleep(20)

In [49]:
print(check_current_id(result_file_path))

1


In [48]:
if __name__ == "__main__":
    main()
    print(quantityOfCorrectAnswer)

open result file 


success
Time request for 60813: 0.49320054054260254, answer:  (3, -10)
